# nb13 — Crooks Ratio Calibration

**Goal:** Apply the Crooks fluctuation theorem to THRML attention-flow processes.  
Derive the theoretical Crooks ratio from canonical Pe parameters and compare against  
the empirical 1.18× GM ratio isolated in nb15.

**Key insight from nb15:** The 26.6× arithmetic ratio decomposes as:  
- 1.18× = GM signal (what the framework predicts — Crooks territory)  
- 22.5× = tail inflation (heavy-tail artifact, not drift signal)

**Crooks theorem (thermodynamic form):**  
$$\frac{P(\sigma = +s)}{P(\sigma = -s)} = e^{s}$$

where $\sigma$ is the dimensionless entropy production. In THRML, we map:  
- Forward protocol: attention captured by void (engagement increasing, c decreasing)  
- Reverse protocol: attention released from void (engagement decreasing, c increasing)  
- Entropy production per interaction: $\sigma = \Delta \text{Pe}$ (Pe as dimensionless transport number)

**Prediction:** $R_{\text{Crooks}} = e^{\Delta\text{Pe}} / e^{0} = e^{\text{Pe}_{\text{eff}}}$  
At ETH canonical: $\text{Pe}_{\text{bull}} = 3.53$, $\text{Pe}_{\text{bear}} = 2.98$  
$\Delta\text{Pe} = 0.55$ → $R = e^{0.55} \approx 1.73×$

But nb15 empirical: 1.18×. Gap factor = 1.73/1.18 ≈ 1.47×  
**This is the calibration target:** find the effective entropy production that matches.

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
from scipy.optimize import brentq
import warnings
warnings.filterwarnings('ignore')

# ── Canonical THRML parameters ──────────────────────────────────────────────
B_ALPHA = 0.867   # bias term (nb07 calibration)
B_GAMMA = 2.244   # coupling term
K = 16            # coordination number (Bethe lattice)
C_CRIT = B_ALPHA / B_GAMMA  # ≈ 0.3865 ... Ising c_crit approximation

def pe(c, k=K, ba=B_ALPHA, bg=B_GAMMA):
    """Canonical Pe formula."""
    return k * np.sinh(2 * (ba - c * bg))

# ── EXP-021C empirical anchors ───────────────────────────────────────────────
C_ETH_BULL = 0.383
C_ETH_BEAR = 0.390
PE_BULL = pe(C_ETH_BULL)   # ≈ 3.53
PE_BEAR = pe(C_ETH_BEAR)   # ≈ 2.98
DELTA_PE = PE_BULL - PE_BEAR

print(f"Canonical THRML parameters:")
print(f"  b_alpha = {B_ALPHA}, b_gamma = {B_GAMMA}, K = {K}")
print(f"  c_crit  = {C_CRIT:.4f}")
print()
print(f"EXP-021C anchors:")
print(f"  Pe_bull = {PE_BULL:.4f} (c_low = {C_ETH_BULL})")
print(f"  Pe_bear = {PE_BEAR:.4f} (c_low = {C_ETH_BEAR})")
print(f"  ΔPe     = {DELTA_PE:.4f}")

In [ ]:
# ── nb15 LogNormal fit parameters ───────────────────────────────────────────
# From nb15: gain distribution LogNormal fit
# Forward (bull→exit, attention captured): μ_fwd, σ_fwd
# Reverse (bear→exit, attention released): μ_rev, σ_rev
# Empirical GM ratio: exp(μ_fwd - μ_rev) = 1.18×

SIGMA_FWD = 3.449    # nb15 result
SIGMA_REV = 2.383    # nb15 result
GM_RATIO  = 1.18     # nb15 empirical GM ratio (arithmetic = 26.6× = 1.18 × 22.5×)
MU_DELTA  = np.log(GM_RATIO)  # ≈ 0.1655 — log-scale GM difference

# For a LogNormal, the mean is exp(μ + σ²/2)
# So arithmetic ratio = GM_ratio × exp((σ_fwd²-σ_rev²)/2)
sigma_tail_factor = np.exp((SIGMA_FWD**2 - SIGMA_REV**2) / 2)
arith_ratio_reconstructed = GM_RATIO * sigma_tail_factor

print(f"nb15 LogNormal parameters:")
print(f"  σ_fwd = {SIGMA_FWD}, σ_rev = {SIGMA_REV}")
print(f"  GM ratio  = {GM_RATIO:.4f}×")
print(f"  Δμ        = {MU_DELTA:.4f}")
print(f"  Tail inflation factor = {sigma_tail_factor:.2f}×")
print(f"  Reconstructed arithmetic ratio = {arith_ratio_reconstructed:.1f}×  (nb15: 26.6×)")
print()

In [ ]:
# ── Crooks fluctuation theorem mapping ─────────────────────────────────────
#
# Standard Crooks:  P(+σ)/P(-σ) = exp(σ)
# 
# THRML mapping:
#   σ_entropy = dimensionless entropy production per attention event
#   In the THRML picture the 'work' done on the system is ΔPe per interaction
#   (the number of nats of bias injected as attention is captured).
#
# Naive prediction: R_Crooks = exp(ΔPe)
# But this is Pe in transport units. To get ratio of GM means we compare:
#   Δμ_theory  = f(c, K) = ?
#
# The connection: Crooks ratio R = exp(ΔF/kT) where ΔF is free-energy difference.
# In THRML: ΔF_eff ∝ Pe (Pe encodes the net drift force).
# But Pe is intensive (per spin per step), while Δμ is extensive over the full trajectory.
# Rescaling: Δμ = Pe_eff × η_efficiency × τ_effective

# Calibrate η × τ from the empirical constraint:
#   Δμ_empirical = 0.1655 = Pe_mean × η_τ
#   Pe_mean = (Pe_bull + Pe_bear)/2 ≈ 3.255

PE_MEAN = (PE_BULL + PE_BEAR) / 2
eta_tau = MU_DELTA / PE_MEAN  # calibration factor

print(f"Crooks calibration:")
print(f"  Naive R_Crooks = exp(ΔPe) = exp({DELTA_PE:.4f}) = {np.exp(DELTA_PE):.4f}×")
print(f"  Empirical GM ratio          = {GM_RATIO:.4f}×")
print(f"  Gap = {np.exp(DELTA_PE)/GM_RATIO:.4f}×")
print()
print(f"  Pe_mean = {PE_MEAN:.4f}")
print(f"  η·τ calibration = Δμ / Pe_mean = {eta_tau:.5f}")
print(f"  Interpretation: η·τ ≈ {eta_tau:.3f} effective time-scale units")
print()

# Verify: R_Crooks_calibrated = exp(Pe_mean × η_τ)
R_calib_check = np.exp(PE_MEAN * eta_tau)
print(f"  Calibrated Crooks (Pe_mean × η·τ): exp({PE_MEAN:.4f}×{eta_tau:.5f}) = {R_calib_check:.4f}×  ✓")

In [ ]:
# ── Crooks ratio as a function of c across the phase diagram ────────────────
#
# For any constraint level c, the expected GM ratio is:
#   R(c) = exp(Pe(c) × η_τ)
# where η_τ is fixed at the ETH calibration value.
#
# This gives us a cross-domain prediction: domains at c=0.3 should have higher
# Crooks ratios than domains at c=0.45.

c_arr = np.linspace(0.10, 0.55, 400)
pe_arr = pe(c_arr)
R_crooks = np.exp(np.clip(pe_arr * eta_tau, -10, 10))  # GM ratio prediction

# Mark c_crit (Pe=0)
c_crit_exact = B_ALPHA / B_GAMMA
pe_zero_c = brentq(lambda c: pe(c), 0.30, 0.45)

print(f"Phase diagram crossing:")
print(f"  Pe=0 at c = {pe_zero_c:.4f}  (b_alpha/b_gamma = {c_crit_exact:.4f})")
print(f"  R_Crooks = 1.0× at c_crit (symmetric, no net drift)")
print()

# Substrate predictions using nb10 c values
substrates = [
    ('AI (unconstrained)', 0.10),
    ('Crypto (speculative)', 0.20),
    ('Gambling', 0.25),
    ('Social Media', 0.30),
    ('ETH (bull)',  C_ETH_BULL),
    ('ETH (bear)',  C_ETH_BEAR),
    ('Crypto (base)', 0.42),
    ('AI (constrained)', 0.45),
    ('JW Org', 0.08),
]

print(f"{'Substrate':<25} {'c':>6} {'Pe':>8} {'R_Crooks':>10}")
print("-" * 55)
for name, c_val in substrates:
    pe_val = pe(c_val)
    r_val = np.exp(pe_val * eta_tau)
    print(f"{name:<25} {c_val:>6.3f} {pe_val:>8.3f} {r_val:>10.3f}×")

In [ ]:
# ── Entropy production rate vs c ────────────────────────────────────────────
#
# From Crooks, the mean entropy production rate is:
#   <σ> = <W_fwd> - ΔF  (Jarzynski)
#
# In THRML terms:
#   <σ> = Pe(c) × η_τ  = Δμ(c)
#
# And the irreversibility (second-law arrow):
#   Σ = <σ> × N_events  (per platform, per day)
#
# We don't have N_events per platform but can show the per-event rate.

sigma_rate = pe_arr * eta_tau  # dimensionless entropy production per event

print("Per-event entropy production at key c values:")
for c_val in [0.10, 0.20, 0.30, pe_zero_c, C_ETH_BULL, 0.45]:
    sr = pe(c_val) * eta_tau
    print(f"  c={c_val:.3f}: σ/event = {sr:.4f} nats")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# FIGURE 1 — Crooks ratio R(c) across the phase diagram
# ══════════════════════════════════════════════════════════════════════════════

FIG_STYLE = {
    'figure.facecolor': '#060810',
    'axes.facecolor':   '#060810',
    'axes.edgecolor':   '#334',
    'text.color':       '#ccd',
    'axes.labelcolor':  '#ccd',
    'xtick.color':      '#889',
    'ytick.color':      '#889',
    'grid.color':       '#1a1f2e',
    'grid.linewidth':   0.5,
}

plt.rcParams.update(FIG_STYLE)

fig, ax = plt.subplots(figsize=(10, 5.6))

# Shade phases
ax.axvspan(0.10, pe_zero_c, alpha=0.08, color='#ff4444', label='Drift regime (Pe > 0)')
ax.axvspan(pe_zero_c, 0.55, alpha=0.06, color='#4488ff', label='Diffusion regime (Pe < 0)')

# R=1 reference
ax.axhline(1.0, color='#ffffff', lw=0.8, ls='--', alpha=0.35, label='R = 1 (symmetric)')

# Crooks ratio curve
mask_pos = pe_arr >= 0
mask_neg = pe_arr < 0
ax.plot(c_arr[mask_pos], R_crooks[mask_pos], color='#ffaa22', lw=2.5, label='R_Crooks (Pe > 0)')
ax.plot(c_arr[mask_neg], R_crooks[mask_neg], color='#6699ff', lw=2.5, ls='-', label='R_Crooks (Pe < 0)')

# Phase boundary
ax.axvline(pe_zero_c, color='#ffffff', lw=1.2, alpha=0.5, ls=':')
ax.text(pe_zero_c + 0.005, 2.5, f'c_crit = {pe_zero_c:.3f}', color='#aab', fontsize=8)

# Empirical anchor
ax.scatter([C_ETH_BULL, C_ETH_BEAR], [GM_RATIO, GM_RATIO],
           c='#ffd700', s=120, zorder=10, label=f'nb15 empirical GM ({GM_RATIO:.2f}×)')
ax.annotate('ETH bull', (C_ETH_BULL, GM_RATIO), (C_ETH_BULL - 0.04, GM_RATIO + 0.08),
            color='#ffd700', fontsize=8, arrowprops=dict(arrowstyle='->', color='#ffd700', lw=0.8))

# Substrate markers
colors_sub = ['#ff2222', '#ff8844', '#ffaa22', '#aaaaff', '#66ccff', '#4488ff', '#ffffff', '#44cc88']
subs_plot = [('AI\n(uncon)', 0.10), ('Crypto\n(spec)', 0.20), ('Gambling', 0.25),
             ('Social\nMedia', 0.30), ('Crypto\n(base)', 0.42), ('AI\n(con)', 0.45)]
for (lbl, c_val), col in zip(subs_plot, colors_sub):
    r_val = np.exp(pe(c_val) * eta_tau)
    ax.scatter([c_val], [r_val], color=col, s=60, zorder=8, alpha=0.9)
    if r_val > 1.05 or r_val < 0.95:
        ax.text(c_val, r_val + 0.04, lbl, color=col, fontsize=6.5, ha='center', va='bottom')

ax.set_xlim(0.08, 0.56)
ax.set_ylim(0.2, 3.0)
ax.set_xlabel('Constraint level c', fontsize=10)
ax.set_ylabel('Crooks GM ratio R(c)', fontsize=10)
ax.set_title('Crooks Fluctuation Ratio vs Constraint Level', fontsize=12, pad=12)
ax.legend(fontsize=7.5, loc='upper right', framealpha=0.25)
ax.grid(True, alpha=0.4)

plt.tight_layout()
plt.savefig('nb13_crooks_ratio_c.svg', format='svg', dpi=150, bbox_inches='tight',
            facecolor='#060810')
plt.close()
print("Saved: nb13_crooks_ratio_c.svg")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# FIGURE 2 — Decomposition: GM signal vs tail inflation
# ══════════════════════════════════════════════════════════════════════════════
#
# Visualise the 26.6× = 1.18× × 22.5× decomposition as stacked bars
# for forward (bull→exit) vs reverse (bear→exit) trajectories.
#
# For a LogNormal(μ, σ): mean = exp(μ + σ²/2)
# Components:
#   GM component:   exp(μ)          → captured by Crooks
#   Tail component: exp(σ²/2)       → irreducible spread (not drift signal)

# Reconstruct μ_fwd from GM ratio and μ_rev=0 (normalised)
mu_fwd = MU_DELTA       # 0.1655
mu_rev = 0.0

gm_fwd  = np.exp(mu_fwd)
gm_rev  = np.exp(mu_rev)  # = 1.0
tail_fwd = np.exp(SIGMA_FWD**2 / 2)
tail_rev  = np.exp(SIGMA_REV**2 / 2)
mean_fwd  = gm_fwd * tail_fwd
mean_rev  = gm_rev * tail_rev
arith_ratio = mean_fwd / mean_rev

print("Decomposition verification:")
print(f"  μ_fwd = {mu_fwd:.4f}, σ_fwd = {SIGMA_FWD}")
print(f"  μ_rev = {mu_rev:.4f}, σ_rev = {SIGMA_REV}")
print(f"  GM_fwd  = {gm_fwd:.4f}")
print(f"  GM_rev  = {gm_rev:.4f}")
print(f"  Tail_fwd = exp(σ_fwd²/2) = {tail_fwd:.2f}")
print(f"  Tail_rev = exp(σ_rev²/2) = {tail_rev:.2f}")
print(f"  Mean_fwd = {mean_fwd:.2f}")
print(f"  Mean_rev = {mean_rev:.2f}")
print(f"  Arithmetic ratio = {arith_ratio:.2f}×  (target: 26.6×)")

fig, axes = plt.subplots(1, 2, figsize=(11, 5.5))
plt.rcParams.update(FIG_STYLE)
for ax in axes:
    ax.set_facecolor('#060810')
    ax.tick_params(colors='#889')

# ── Left: stacked bar decomposition ─────────────────────────────────────────
ax = axes[0]
labels = ['Forward (bull→exit)', 'Reverse (bear→exit)']
gm_vals   = [gm_fwd, gm_rev]
tail_vals = [tail_fwd, tail_rev]

x = np.array([0, 1])
b1 = ax.bar(x, gm_vals, color=['#ffaa22','#6699ff'], alpha=0.85,
            label='GM component exp(μ)', width=0.5)
b2 = ax.bar(x, tail_vals, bottom=gm_vals, color=['#ff6622','#3366cc'],
            alpha=0.7, label='Tail inflation exp(σ²/2)', width=0.5, hatch='..')

# Annotate totals
for xi, (gv, tv, lbl) in enumerate(zip(gm_vals, tail_vals, ['Mean = 26.4', 'Mean = 1.0'])):
    ax.text(xi, gv + tv + 0.5, f'{gv+tv:.1f}', ha='center', color='#ccd', fontsize=9)

ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=9)
ax.set_ylabel('Mean value (normalised to reverse GM=1)', fontsize=9)
ax.set_title('LogNormal Decomposition:\nGM Signal vs Tail Inflation', fontsize=10)
ax.legend(fontsize=8, framealpha=0.25)
ax.grid(axis='y', alpha=0.4)
ax.set_ylim(0, 32)

# Annotate ratio
ax.annotate('', xy=(1.25, gm_rev + tail_rev/2), xytext=(1.25, gm_fwd + tail_fwd/2),
            arrowprops=dict(arrowstyle='<->', color='#ff8800', lw=1.5))
ax.text(1.32, (gm_fwd + tail_fwd/2 + gm_rev + tail_rev/2) / 2,
        f'{arith_ratio:.1f}×', color='#ff8800', fontsize=10, va='center')

# ── Right: GM ratio vs σ_fwd at fixed σ_rev ─────────────────────────────────
ax = axes[1]
sigma_fwd_range = np.linspace(0.5, 5.0, 200)
# Arithmetic ratio = GM_ratio × exp((σ_fwd² - σ_rev²)/2)
# For GM_ratio = 1.18, how does arithmetic ratio scale with σ_fwd?
arith_ratios_scan = GM_RATIO * np.exp((sigma_fwd_range**2 - SIGMA_REV**2) / 2)

ax.plot(sigma_fwd_range, arith_ratios_scan, color='#ffaa22', lw=2.5)
ax.axvline(SIGMA_FWD, color='#ffd700', lw=1.5, ls='--', alpha=0.8)
ax.axhline(GM_RATIO, color='#6699ff', lw=1.2, ls='--', alpha=0.7,
           label=f'GM ratio = {GM_RATIO}× (Crooks signal)')
ax.scatter([SIGMA_FWD], [arith_ratio], color='#ffd700', s=120, zorder=10,
           label=f'nb15 empirical (σ_fwd={SIGMA_FWD})')

ax.text(SIGMA_FWD + 0.05, arith_ratio + 0.5,
        f'{arith_ratio:.1f}×', color='#ffd700', fontsize=9)

# Shade the "tail-dominated" zone
ax.fill_between(sigma_fwd_range, GM_RATIO, arith_ratios_scan,
                where=arith_ratios_scan > GM_RATIO,
                alpha=0.15, color='#ff4444', label='Tail inflation (not drift)')

ax.set_xlabel('σ_fwd (forward trajectory spread)', fontsize=9)
ax.set_ylabel('Arithmetic gain ratio', fontsize=9)
ax.set_title('Arithmetic Ratio vs Tail Width\n(σ_rev fixed at nb15 value)', fontsize=10)
ax.set_yscale('log')
ax.set_ylim(0.5, 1000)
ax.legend(fontsize=8, framealpha=0.25)
ax.grid(True, alpha=0.4)

plt.tight_layout()
plt.savefig('nb13_decomposition.svg', format='svg', dpi=150, bbox_inches='tight',
            facecolor='#060810')
plt.close()
print("Saved: nb13_decomposition.svg")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# FIGURE 3 — Calibration: η·τ as a free parameter
# ══════════════════════════════════════════════════════════════════════════════
#
# We know R_empirical = 1.18× at ETH (c≈0.386).
# The Crooks mapping is: Δμ = Pe(c) × η_τ
# So η_τ = Δμ / Pe = 0.1655 / 3.255 ≈ 0.0508
#
# This figure shows:
# 1. η_τ as a function of c (what you'd infer if you *knew* R_empirical = 1.18× at each c)
# 2. The sensitivity: ∂Δμ/∂c = η_τ × dPe/dc

# η_τ from ETH calibration (single point)
ETA_TAU = eta_tau
print(f"Calibration: η·τ = {ETA_TAU:.5f}")

# What η_τ would be inferred if R=1.18× were observed at each c?
eta_tau_c = np.where(np.abs(pe_arr) > 0.01,
                     MU_DELTA / pe_arr,
                     np.nan)

# dΔμ/dc at fixed η_τ = η_τ × dPe/dc
# dPe/dc = -2·K·B_GAMMA·cosh(2·(B_ALPHA - c·B_GAMMA))
dpe_dc = -2 * K * B_GAMMA * np.cosh(2 * (B_ALPHA - c_arr * B_GAMMA))
dmu_dc = ETA_TAU * dpe_dc

fig, axes = plt.subplots(1, 2, figsize=(11, 5))
for ax in axes:
    ax.set_facecolor('#060810')
    ax.tick_params(colors='#889')

# Left: η_τ(c) — would be constant if Crooks perfectly maps Pe
ax = axes[0]
mask_valid = np.abs(pe_arr) > 0.2
ax.plot(c_arr[mask_valid & mask_pos], eta_tau_c[mask_valid & mask_pos],
        color='#ffaa22', lw=2)
ax.plot(c_arr[mask_valid & mask_neg], eta_tau_c[mask_valid & mask_neg],
        color='#6699ff', lw=2)
ax.axhline(ETA_TAU, color='#ffd700', lw=1.5, ls='--',
           label=f'Calibrated η·τ = {ETA_TAU:.4f} (ETH anchor)')
ax.axvline(pe_zero_c, color='#ffffff', lw=0.8, ls=':', alpha=0.4)
ax.scatter([C_ETH_BULL], [ETA_TAU], color='#ffd700', s=120, zorder=10)

ax.set_xlim(0.10, 0.55)
ax.set_ylim(-0.20, 0.20)
ax.set_xlabel('Constraint level c', fontsize=9)
ax.set_ylabel('η·τ inferred from R=1.18× at each c', fontsize=9)
ax.set_title('Crooks Efficiency η·τ\n(constant if mapping is exact)', fontsize=10)
ax.legend(fontsize=8, framealpha=0.25)
ax.grid(True, alpha=0.4)
ax.text(0.25, 0.14,
        'If η·τ were constant across c,\nCrooks ratio would perfectly predict\nGM ratio at all constraint levels.',
        color='#aab', fontsize=7.5, va='top',
        bbox=dict(boxstyle='round', fc='#111', ec='#334', alpha=0.8))

# Right: sensitivity dΔμ/dc
ax = axes[1]
ax.plot(c_arr, dmu_dc, color='#ff6688', lw=2)
ax.axhline(0, color='#ffffff', lw=0.8, ls='--', alpha=0.3)
ax.axvline(pe_zero_c, color='#ffffff', lw=0.8, ls=':', alpha=0.4)

# Mark ETH point
dmu_eth = ETA_TAU * (-2 * K * B_GAMMA * np.cosh(2 * (B_ALPHA - C_ETH_BULL * B_GAMMA)))
ax.scatter([C_ETH_BULL], [dmu_eth], color='#ffd700', s=120, zorder=10,
           label=f'ETH: ∂Δμ/∂c = {dmu_eth:.3f}')
ax.set_xlabel('Constraint level c', fontsize=9)
ax.set_ylabel('∂Δμ/∂c (sensitivity of GM ratio to constraint)', fontsize=9)
ax.set_title('GM Ratio Sensitivity\ndΔμ/dc = η·τ × dPe/dc', fontsize=10)
ax.legend(fontsize=8, framealpha=0.25)
ax.grid(True, alpha=0.4)
ax.text(0.30, -3.5,
        f'At ETH (c={C_ETH_BULL:.3f}):\n∂Δμ/∂c ≈ {dmu_eth:.3f}\n→ 1pp relaxation in c\n→ {abs(dmu_eth)*0.01*100:.1f}% shift in log-ratio',
        color='#ffd700', fontsize=7.5, va='top',
        bbox=dict(boxstyle='round', fc='#111', ec='#ffd70044', alpha=0.85))

plt.tight_layout()
plt.savefig('nb13_calibration.svg', format='svg', dpi=150, bbox_inches='tight',
            facecolor='#060810')
plt.close()
print("Saved: nb13_calibration.svg")

In [ ]:
# ── Summary ──────────────────────────────────────────────────────────────────

print("="*60)
print("nb13 — Crooks Ratio Calibration — SUMMARY")
print("="*60)
print()
print("THEOREM APPLICATION:")
print("  Crooks: P(+σ)/P(-σ) = exp(σ)")
print(f"  THRML mapping: σ = Pe(c) × η·τ")
print(f"  → GM ratio R(c) = exp(Pe(c) × η·τ)")
print()
print("CALIBRATION (ETH anchor from nb15):")
print(f"  Empirical GM ratio = {GM_RATIO:.3f}×  (Δμ = {MU_DELTA:.4f})")
print(f"  Pe_mean at ETH    = {PE_MEAN:.4f}")
print(f"  η·τ calibrated    = {ETA_TAU:.5f}")
print()
print("KEY RESULT:")
print(f"  Arithmetic 26.6× = {GM_RATIO:.2f}× (Crooks/GM) × {sigma_tail_factor:.1f}× (tail)")
print(f"  Crooks operates on GM ratio only. Tail inflation = separate physics.")
print()
print("CROSS-DOMAIN PREDICTIONS (η·τ = const):")
for name, c_val in substrates:
    pe_val = pe(c_val)
    r_val = np.exp(pe_val * ETA_TAU)
    arrow = '← HIGH drift' if r_val > 1.5 else ('← near-symmetric' if abs(r_val-1) < 0.1 else '')
    print(f"  {name:<25} R = {r_val:.3f}×  {arrow}")
print()
print("SVGs generated: nb13_crooks_ratio_c.svg, nb13_decomposition.svg, nb13_calibration.svg")